# Classiﬁcation : Convolutional Neural Networks  

In [ ]:
# Import libraries
import keras
import os
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense, Dropout,BatchNormalization, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [ ]:
#Import data
train_data = pd.read_csv("/kaggle/input/train.csv")
num_classes=5

X = train_data[train_data.columns[2:786]].values
X=X/255
Y = np.ravel(train_data[['Label']])
Y = keras.utils.to_categorical(Y,num_classes)

X_train,X_val,y_train,y_val = train_test_split(X,Y,test_size = 0.1,random_state = 42)

testing_data= pd.read_csv("/kaggle/input/testX.csv")
testing_data=testing_data[testing_data.columns[1:785]].values

testing_data =testing_data/255

#Reshaping the data
testing_data = testing_data.reshape(testing_data.shape[0], 28, 28, 1)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)


In [ ]:
#Save checkpoint
save_dir = os.path.join(os.getcwd(), 'saved_model')
model_name = 'fmnist_resnet_model.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir,model_name)
print(filepath)

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

checkpoint = ModelCheckpoint(filepath=filepath,
                             verbose=1,
                             save_best_only=True)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
callbacks = [checkpoint, lr_reducer]



##### CNN1

In [ ]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, (3,3), input_shape = (28,28, 1),padding='same', activation = 'relu'))
classifier.add(Convolution2D(32, (3,3),padding='same', activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2),strides=2))
classifier.add(Dropout(0.2))

classifier.add(Convolution2D(64, (3,3), activation = 'relu'))
classifier.add(Convolution2D(64, (3,3),activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2),strides=2))

classifier.add(Dropout(0.3))

classifier.add(Flatten())


classifier.add(Dense(activation = 'relu',units=256))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.7))

classifier.add(Dense(activation = 'softmax',units=5))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.summary()#---------------->88.04%

In [ ]:
classifier.fit(X_train, y_train,
              batch_size=256,
              epochs=30,
               verbose=1,
              validation_data=(X_val,y_val))

In [ ]:
test_loss, test_acc = classifier.evaluate(X_val,  y_val, verbose=2)

print('\nTest accuracy:', test_acc)

##### CNN2

In [ ]:
cnn2 = Sequential([
    Convolution2D(8, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(28,28, 1)),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Convolution2D(16, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Convolution2D(32, kernel_size=(5,5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(5, activation='softmax')
])
cnn2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
cnn2.summary()

In [ ]:
cnn2.fit(X_train, y_train,
              batch_size=256,
              epochs=30,
               verbose=1,
              validation_data=(X_val,y_val))

In [ ]:
test_loss, test_acc = cnn2.evaluate(X_val,  y_val, verbose=2)

print('\nTest accuracy of CNN2:', test_acc)

##### CNN3

In [ ]:
cnn3 = Sequential([
    Convolution2D(32, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(28,28,1)),
    Convolution2D(32, kernel_size=(3, 3), activation='relu',padding='same'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Dropout(0.2),
    Convolution2D(64, kernel_size=(3, 3), activation='relu'),
    Convolution2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Dropout(0.3),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(5, activation='softmax')
])
cnn3.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
cnn3.summary()#----------BEST AT 30 EPOC

In [ ]:
fashion_train =cnn3.fit(X_train, y_train,
              batch_size=256,
              epochs=40,
              verbose=1,
              validation_data=(X_val,y_val))

In [ ]:
test_eval = cnn3.evaluate(X_val, y_val, verbose=2)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])
#test_loss, test_acc = cnn3.evaluate(X_val,  y_val, verbose=2)

#print('\nTest accuracy of CNN2:', test_acc)


In [ ]:
import matplotlib.pyplot as plt
accuracy = fashion_train.history['accuracy']
val_accuracy = fashion_train.history['val_accuracy']
loss = fashion_train.history['loss']
val_loss = fashion_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

##### CNN4

In [ ]:
cnn4 = Sequential([
    Convolution2D(filters=32, kernel_size=5, activation='relu',padding='same',input_shape=(28,28,1)),
    MaxPooling2D(pool_size=2),
    Convolution2D(filters=64, kernel_size=3, activation='relu',padding='same'),
    MaxPooling2D(pool_size=2),
    Convolution2D(128, kernel_size=1, activation='relu',padding='same'),
    #MaxPooling2D(pool_size=(2, 2),strides=2),
    Dropout(0.2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.25),
    Dense(32, activation='relu'),
    #Dropout(0.3),
    Dense(5, activation='softmax')
])
cnn4.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
cnn4.summary()

In [ ]:
cnn4.fit(X_train, y_train,
              batch_size=1024,
              epochs=400,
               verbose=1,
              validation_data=(X_val,y_val))

In [ ]:
test_loss, test_acc = cnn4.evaluate(X_val,  y_val, verbose=2)

print('\nTest accuracy of CNN4:', test_acc)

In [ ]:
ynew = cnn3.predict_classes(testing_data)
print(ynew)
np.savetxt("foo.csv", ynew,fmt='%d', delimiter=",")

In [ ]:
cnn11 = Sequential([
    Convolution2D(8, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Convolution2D(16, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Convolution2D(32, kernel_size=(5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Flatten(),
    Dense(120, activation='relu'),
    Dropout(0.2),
    Dense(84, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')
])
cnn11.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
cnn11.summary()

In [ ]:
cnn11.fit(X_train, y_train,
              batch_size=256,
              epochs=300,
               verbose=1,
              validation_data=(X_val,y_val))

In [ ]:
test_loss, test_acc = cnn11.evaluate(X_val,  y_val, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:

ynew = cnn2.predict_classes(testing_data)
print(ynew)
np.savetxt("foo.csv", ynew,fmt='%d', delimiter=",")

Resnet

In [ ]:
from keras.layers import  Conv2D, Activation
from keras.layers import MaxPooling2D, AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from sklearn.model_selection import train_test_split

epochs = 100
batch_size = 32
data_augmentation = False
img_size = 28

num_classes = 5
num_filters = 64
num_blocks = 4
num_sub_blocks = 2
use_max_pool = False

#Creating model based on ResNet published archietecture
inputs = Input(shape=(28,28,1))
x = Convolution2D(num_filters, padding='same', 
           kernel_initializer='he_normal', 
           kernel_size=7, strides=2,
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#Check by applying max pooling later (setting it false as size of image is small i.e. 28x28)
if use_max_pool:
    x = MaxPooling2D(pool_size=3,padding='same', strides=2)(x)
    num_blocks =3
#Creating Conv base stack 

# Instantiate convolutional base (stack of blocks).
for i in range(num_blocks):
    for j in range(num_sub_blocks):
        strides = 1
        is_first_layer_but_not_first_block = j == 0 and i > 0
        if is_first_layer_but_not_first_block:
            strides = 2
        #Creating residual mapping using y
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   strides=strides,
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(y)
        y = BatchNormalization()(y)
        if is_first_layer_but_not_first_block:
            x = Conv2D(num_filters,
                       kernel_size=1,
                       padding='same',
                       strides=2,
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(x)
        #Adding back residual mapping
        x = keras.layers.add([x, y])
        x = Activation('relu')(x)

    num_filters = 2 * num_filters

# Add classifier on top.
x = AveragePooling2D()(x)
y = Flatten()(x)
outputs = Dense(num_classes,
                activation='softmax',
                kernel_initializer='he_normal')(y)

# Instantiate and compile model.
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()